# 我想要实现的是流式解码的时候支持多个停止符, 就是支持列表形式, 可以是字符串或者是 token_id

In [1]:
# #模型下载
# from modelscope import snapshot_download
# model_dir = snapshot_download('qwen/Qwen1.5-0.5B-Chat')
# print(model_dir)

In [17]:
# 加载下 qwen 的分词器
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

model_dir = r"D:\code\pretrained_model\modelscope\hub\qwen\Qwen1___5-0___5B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
tokenizer

c:\Anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Qwen2TokenizerFast(name_or_path='D:\code\pretrained_model\modelscope\hub\qwen\Qwen1___5-0___5B-Chat', vocab_size=151643, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.device, model.dtype

(device(type='cuda', index=0), torch.float16)

In [4]:
prompt = "Give me a short introduction to large language model."
prompt = "你是谁"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


In [5]:
print(len(tokenizer.encode(response)))
print(response)

37
我是来自阿里云的大规模语言模型，我叫通义千问。我能够回答问题、创作文字，还能表达观点、撰写代码。有什么我可以帮助你的吗？


# 我需要实现一个函数, 在模型流式输出的时候, 支持多个停止符, 停止符可以是字符串或者是数字

In [18]:
output_ids = tokenizer.encode("我是通义千问, 是来自阿里云的超大规模语言模型")
print(len(output_ids))
print(output_ids)

for output_id in output_ids:
    print(tokenizer.decode(output_id), output_id)

15
[104198, 31935, 64559, 99320, 56007, 11, 54851, 101919, 102661, 99718, 9370, 71304, 105483, 102064, 104949]
我是 104198
通 31935
义 64559
千 99320
问 56007
, 11
 是 54851
来自 101919
阿里 102661
云 99718
的 9370
超 71304
大规模 105483
语言 102064
模型 104949


In [20]:
output_chunk_list = [tokenizer.decode(output_id) for output_id in output_ids]
print(output_chunk_list)

['我是', '通', '义', '千', '问', ',', ' 是', '来自', '阿里', '云', '的', '超', '大规模', '语言', '模型']


In [10]:
class Env:
    """用来保存解码过程中的临时变量, 解码是流式解码, 每次给一个 output_id"""
    def __init__(self):
        # 保存先前未输出的文本
        self.previous = ""
        # 当前收到的 id 对应的文本
        self.current = ""
        # 是否停止
        self.stop = False

In [9]:
def decode_one(output_id: int, env: Env, stop_list: list[int|str]):
    """解码一个 output_id, 这里只考虑 stop_list 为文本的情况"""
    stop_str_list = [x for x in stop_list if isinstance(x, str) and len(x) > 0]
    # 将当前 id 解码成文本. TODO: 没有考虑多个 id 对应一个文本的情况
    current = tokenizer.decode(output_id)
    
    # 先拼接上历史的文本
    current = env.previous + current

    # 检查是否可能有停止符
    # 什么情况下是安全的, 可以直接输出 current



In [21]:
def check_stop_symbols(stream, stop_symbols):
    for word in stream:
        if any(symbol in word for symbol in stop_symbols):
            break
        yield word

# 使用方法：
stream = ['我是', '通', '义', '千', '问', ',', ' 是', '来自', '阿里', '云', '的', '超', '大规模', '语言', '模型']
stop_symbols = ['通义']
for chunk in check_stop_symbols(stream, stop_symbols):
    print(chunk)

我是
通
义
千
问
,
 是
来自
阿里
云
的
超
大规模
语言
模型


In [58]:
# TODO: 感觉有些复杂, 也不知道实现的对不对, 这是我和 bing 的混合结果
# 定义输入和停止符列表
input = ['我是', '通', '义', '千', '问', ',', ' 是', '来自', '阿里', '云', '的', '超', '大规模', '语言', '模型']
stop_words = ['通1义', "义千"]

# 定义一个函数，用于在流式输出的字符串中检查是否遇到了停止符
def check_stop_words(input, stop_words):
  # 初始化一个空字符串，用于存储输出
  output = ""
  # 初始化一个空字符串，用于存储当前的候选停止符
  candidate = ""
  # 遍历输入的字符串列表
  for word in input:
    if word in stop_words:
        output += candidate
        if output:
            yield output
        output = ""
        print("==遇到停止符", word)
        break
    # 如果当前的候选停止符不为空，就将当前的字符串添加到候选停止符中
    if candidate:
      candidate += word
    # 如果当前的字符串是停止符列表中的第一个字符，就将当前的字符串作为候选停止符
    elif word in [s[0] for s in stop_words]:
      candidate = word
    # 否则，将当前的字符串添加到输出中，并加上一个空格
    else:
      output += word + " "
      yield output
      output = ""
    # 如果当前的候选停止符是停止符列表中的一个，就停止输出，并返回结果
    if candidate in stop_words:
      print("==遇到停止符", candidate)
      break
    # 还有可能是结尾匹配的
    for stop_word in stop_words:
        if candidate.endswith(stop_word):
            output += candidate[:-len(stop_word)]
            candidate = ""
            yield output
            print("--遇到了结尾匹配的", stop_word)
            return
    # 注意, 虽然第一个字符串在停止符列表中, 但是它不是停止符, 所以需要将它加回来
    if candidate and len(candidate) >= max(map(len, stop_words)) and candidate not in stop_words:
      print("--遇到了无法匹配的", candidate)
      output += candidate
      candidate = ""

  # 返回输出
#   return output

# 调用函数，并打印结果
for  x in check_stop_words(input, stop_words):
    print(x)

我是 
通
--遇到了结尾匹配的 义千


实现一个python算法, 在流式输出的字符串中检查是否遇到了停止符列表, 如果遇到了, 就停止输出, 停止符是字符串, 可能有多个字符组成, 也可能是单个字符.
比如输入是 ['我是', '通', '义', '千', '问', ',', ' 是', '来自', '阿里', '云', '的', '超', '大规模', '语言', '模型'], 停止符列表是 ['通义'].
这时候就应该只输出 "我是", 因为后面遇到了 "通义" 这个停止符. 不能只判断当前字符是否是停止符, 因为可能停止符是多个字符组成的.